In [5]:
import pandas as pd
import statistics
import heapq


mens_data = pd.read_csv("mens_gym_data_fil.csv")
womens_data = pd.read_csv("womens_gym_data_fil.csv")
mens_data["Month"] = mens_data["Date"].apply(lambda x: x.split(" ")[-2])
mens_data['Month'] = mens_data['Month'].replace({'Jan': 1, 'Feb': 2, 'Mar': 3, 
                                   'Apr': 4, 'May': 5, 'June': 6, 'July': 7, 'Aug': 8, 
                                   'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})
womens_data["Month"] = womens_data["Date"].apply(lambda x: x.split(" ")[-2])
womens_data['Month'] = womens_data['Month'].replace({'Jan': 1, 'Feb': 2, 'Mar': 3, 
                                   'Apr': 4, 'May': 5, 'June': 6, 'July': 7, 'Aug': 8, 
                                   'Sept': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12})

mens_data['name'] = mens_data['name'].replace('Fred RICHARD', "Frederick Nathaniel RICHARD")
mens_data['FirstName'] = mens_data['FirstName'].replace('Fred', 'Frederick')
mens_data['FirstName'] = mens_data['FirstName'].replace('Frederick Nathaniel', 'Frederick')
mens_data['name'] = mens_data['name'].replace('Frederick RICHARD', "Frederick Nathaniel RICHARD")
mens_data['name'] = mens_data['name'].replace('Joshua KARNES', "Joshua Andrew KARNES")
mens_data['FirstName'] = mens_data['FirstName'].replace("Joshua Andrew", 'Joshua')
mens_data['name'] = mens_data['name'].replace("Curran Michael PHILLIPS", "Curran PHILLIPS")
mens_data['FirstName'] = mens_data['FirstName'].replace("Curran Michael", 'Curran')
mens_data['FirstName'] = mens_data['FirstName'].replace("Taylor Troy", 'Taylor')
mens_data['name'] = mens_data['name'].replace("Taylor Troy CHRISTOPULOS", "Taylor CHRISTOPULOS")
mens_data['name'] = mens_data['name'].replace('Khoi Alexander YOUNG', 'Khoi YOUNG')
mens_data['FirstName'] = mens_data['FirstName'].replace("Khoi Alexander", 'Khoi')
mens_data['name'] = mens_data['name'].replace('Ian Hunter SKIRKEY', 'Ian SKIRKEY')
mens_data['FirstName'] = mens_data['FirstName'].replace("Ian Hunter", 'Ian')
mens_data['name'] = mens_data['name'].replace('Shane Michael WISKUS', 'Shane WISKUS')
mens_data['FirstName'] = mens_data['FirstName'].replace("Shane Michael", 'Shane')
mens_data['name'] = mens_data['name'].replace('Yul Kyung Tae MOLDAUER', 'Yul MOLDAUER')
mens_data['FirstName'] = mens_data['FirstName'].replace("Yul Kyung Tae", 'Yul')

womens_data['name'] = womens_data['name'].replace('Joscelyn Michelle ROBERSON', 'Joscelyn ROBERSON')
womens_data['FirstName'] = womens_data['FirstName'].replace("Joscelyn Michelle", 'Joscelyn')

womens_data['name'] = womens_data['name'].replace('Nola Rhianne MATTHEWS', 'Nola MATTHEWS')
womens_data['FirstName'] = womens_data['FirstName'].replace("Nola Rhianne", 'Nola')


mens_data.to_csv('mens_gym_data_fil_2.csv')
womens_data.to_csv('womens_gym_data_fil_2.csv')
frames = [mens_data, womens_data]
result = pd.concat(frames)
result.to_csv('filtered_3.csv')

set(result['FirstName'])



{'Addison',
 'Adriel',
 'Aidan',
 'Alex',
 'Alicia',
 'Amari',
 'Amelia',
 'Asher',
 'Ashlee',
 'Ashton',
 'Avery',
 'Blake',
 'Brandon',
 'Brigham',
 'Brody',
 'Brooke',
 'Caden',
 'Cailen',
 'Cameron',
 'Carson',
 'Charlie',
 'Charlotte',
 'Chase',
 'Christopher',
 'Colin',
 'Colt',
 'Connor',
 'Crew',
 'Curran',
 'Dallas',
 'Daniel',
 'David',
 'Donnell',
 'Dulcy',
 'Elle',
 'Ethan',
 'Evan',
 'Eveylynn',
 'Frederick',
 'Fuzzy',
 'Garrett',
 'Hunter',
 'Ian',
 'Isaiah',
 'J.R.',
 'Jack',
 'Jackson',
 'Jade',
 'Javier',
 'Jeremy',
 'Johnny',
 'Jordan',
 'Jordis',
 'Joscelyn',
 'Joshua',
 'Kaliya',
 'Kameron',
 'Katelyn',
 'Kayla',
 'Kazuki',
 'Kelise',
 'Kellen',
 'Kelton',
 'Khoi',
 'Kristian',
 'Kyle',
 'Landen',
 'Landon',
 'Leanne',
 'Leo',
 'Levi',
 'Lexi',
 'Luke',
 'Mac',
 'Madray',
 'Marissa',
 'Matt',
 'Matthew',
 'Maxim',
 'Maxwell',
 'Michael',
 'Michelle',
 'Mike',
 'Myli',
 'Nicky',
 'Noah',
 'Nola',
 'Oliver',
 'Parker',
 'Patrick',
 'Paul',
 'Riley',
 'Ronan',
 'Sam',


In [45]:
def weight_adjustment(month, year, roundtype):
    weight = 1
    if roundtype in ["AAfinal", "TeamFinal", "final"]:
        weight += 0.5
    if year == 2022:
        weight = 1 + 0.0625 * (month - 1)
    else:
        weight = 1.75 + 0.0625 * (month - 1)
        
    return weight

def adjusted_score_c(data_point):
    adjusted_score = 14
    rank = data_point["Rank"]
    if rank == 1:
        adjusted_score *= 10
    elif rank == 2:
        adjusted_score *= 7.5
    elif rank == 3:
        adjusted_score *= 5
    elif rank <= 5:
        adjusted_score *= 3
    elif rank <= 10:
        adjusted_score *= 2
    elif pd.isna(rank):
        adjusted_score *= 1
    
    return adjusted_score

In [140]:
mens_apps = set(mens_data['Apparatus'])

## Starting with the male performances
## 1. I would like to put recent performances with more weights.
## 2. I would like to weight top medals with higher weights.
## 3. I would like to weight final round performances higher.
## 4. I would like to weight consistency more. -> This'll come in for determining our best guys OR as a smell check
## 5. If they don't rank in a particular event, I heavily weigh against.


## Structure of the Dict
## {Abe: [(AppA: WeightedScore), (AppB: WeightedScore), ...]

## WeightedScore calculated by ListOfScores: [AdjustedScore, WeightGiven]

## WeightGiven
## Recent Performances = 1 - 1.75 if 2022, 1.75 - 2.5 if 2023
## Round Type = 1 if not final, 1.5 if final

## AdjustedScore
## TopMedal = 2 if Gold, 1.75 if Silver, 1.5 if Bronze, 1.25 if 4th or 5th
## RankingBad = 0.6x if <30th, 0.85x if <20th

## Measure Consistency at the End
def create_performances(mens_data):
    male_performances = {}
    males_sw = {}
    for i in set(mens_data['name']):
        person_data = mens_data[mens_data["name"] == i].copy()
        performances = []
        score_and_weights = []
        for j in set(mens_data['Apparatus']):
            rel_data = person_data[person_data["Apparatus"] == j].copy()
            total_score = 0
            total_weights = 0
            for k in range(0, len(rel_data)):
                rel_point = rel_data.iloc[k]
                weightgiven = weight_adjustment(rel_point["Month"], rel_point["Year"], rel_point["Round"])
                adjusted_score = adjusted_score_c(rel_point)
                score_and_weights.append((weightgiven, adjusted_score, j, rel_point['Score'], rel_point['Rank']))
                total_score += weightgiven * adjusted_score
                total_weights += weightgiven
            if len(rel_data) == 0:
                performances.append((j, 0))
            else:
                performances.append((j, total_score / total_weights))
        males_sw[i] = score_and_weights
        male_performances[i] = performances
    return male_performances, males_sw
            
    
def create_sum_avgs(mens_data):
    male_performances, males_swe = create_performances(mens_data)
    male_sum_scores = []
    male_avg_scores = []
    male_best1_scores = []
    male_best2_scores = []
    male_best3_scores = []
    for m in set(mens_data['name']):
        personsscores = male_performances[m]
        sc = [s[1] for s in personsscores]
        male_sum_scores.append((m, sum(sc)))
        stripped = []
        for i in sc:
            if i > 8:
                stripped.append(i)
        if len(stripped) == 0:
            continue
        ## Best 1
        male_best1_scores.append((m, max(stripped)))
        
        ## Best 2
        if len(stripped) >= 2:
            male_best2_scores.append((m, statistics.mean(heapq.nlargest(2, stripped))))
        
        ## Best 3
        if len(stripped) >= 3:
            male_best3_scores.append((m, statistics.mean(heapq.nlargest(3, stripped))))
    
    return male_sum_scores, males_swe, male_performances, [male_best1_scores, 
                                                                            male_best2_scores, male_best3_scores]          
            
male_sum_scores, males_swe, male_performances, best_scores = create_sum_avgs(mens_data)

In [141]:
ms1 = sorted(male_sum_scores, key=lambda x: x[1], reverse = True)
ms2 = sorted(best_scores[0], key=lambda x: x[1], reverse = True)
ms3 = sorted(best_scores[1], key=lambda x: x[1], reverse = True)
ms4 = sorted(best_scores[2], key=lambda x: x[1], reverse = True)
ms_combined = set([a[0] for a in ms1[0:15]] + [a[0] for a in ms2[0:15]] 
                  + [a[0] for a in ms3[0:15]] + [a[0] for a in ms4[0:15]])

In [142]:
ranks = {}
avg_scores = []
for i in set([a[0] for a in ms1]):
    k = 1
    avg = 0
    ranks_curr = []
    while ms1[k - 1][0] != i and k < len(ms1):
        k += 1
    ranks_curr.append(k)
    avg += k
    
    k = 1
    while ms2[k - 1][0] != i and k < len(ms2):
        k += 1
    ranks_curr.append(k)
    avg += k
    k = 1
    while ms3[k - 1][0] != i and k < len(ms3):
        k += 1
    ranks_curr.append(k)
    avg += k
    
    k = 1
    while ms4[k - 1][0] != i and k < len(ms4):
        k += 1
    ranks_curr.append(k)
    avg += k
    
    ranks[i] = ranks_curr
    avg_scores.append((float(avg/4), i))

In [143]:
sorted(avg_scores)[0:20]

[(1.0, 'Brody MALONE'),
 (2.75, 'Donnell WHITTENBURG'),
 (4.75, 'Curran PHILLIPS'),
 (4.75, 'Frederick Nathaniel RICHARD'),
 (5.5, 'Khoi YOUNG'),
 (5.75, 'Taylor BURKHART'),
 (7.5, 'Asher HONG'),
 (8.5, 'Colt WALKER'),
 (9.25, 'Paul JUDA'),
 (11.25, 'Joshua Andrew KARNES'),
 (11.75, 'Riley LOOS'),
 (14.75, 'Shane WISKUS'),
 (14.75, 'Yul MOLDAUER'),
 (16.75, 'Cameron BOCK'),
 (17.0, 'Ian LASIC-ELLIS'),
 (17.25, 'Brandon BRIONES'),
 (19.5, 'Michael JAROH'),
 (21.75, 'Matt CORMIER'),
 (22.5, 'Javier ALFONSO'),
 (24.25, 'Zachary NUNEZ')]

In [130]:
print([(a, ranks[a]) for a in ms_combined])

[('Riley LOOS', [10, 16, 10, 11]), ('Ian LASIC-ELLIS', [14, 20, 18, 16]), ('Alex DIAB', [56, 12, 17, 19]), ('Cameron BOCK', [12, 24, 16, 15]), ('Yul MOLDAUER', [15, 19, 12, 13]), ('Shane WISKUS', [11, 21, 15, 12]), ('Colt WALKER', [8, 8, 9, 9]), ('Blake SUN', [55, 14, 14, 18]), ('Khoi YOUNG', [7, 7, 4, 4]), ('Stephen NEDOROSCIK', [64, 4, 89, 80]), ('Taylor BURKHART', [5, 6, 5, 7]), ('Brandon BRIONES', [32, 10, 13, 14]), ('Connor MCCOOL', [75, 13, 21, 80]), ('Donnell WHITTENBURG', [2, 3, 3, 3]), ('Asher HONG', [4, 11, 7, 8]), ('Joshua Andrew KARNES', [9, 15, 11, 10]), ('Paul JUDA', [6, 17, 8, 6]), ('Frederick Nathaniel RICHARD', [3, 5, 6, 5]), ('Brody MALONE', [1, 1, 1, 1]), ('Curran PHILLIPS', [13, 2, 2, 2]), ('Ian SKIRKEY', [79, 9, 89, 80])]


In [144]:
len(ms_combined)

21

In [105]:
len(set(mens_data['name']))

97

In [145]:
womens_apps = set(womens_data['Apparatus'])
womens_sum_scores, womens_swe, womens_performances, womens_best_scores = create_sum_avgs(womens_data)

In [146]:
ms1 = sorted(womens_sum_scores, key=lambda x: x[1], reverse = True)
ms2 = sorted(womens_best_scores[0], key=lambda x: x[1], reverse = True)
ms3 = sorted(womens_best_scores[1], key=lambda x: x[1], reverse = True)
ms4 = sorted(womens_best_scores[2], key=lambda x: x[1], reverse = True)
ms_combined = set([a[0] for a in ms1[0:15]] + [a[0] for a in ms2[0:15]] 
                  + [a[0] for a in ms3[0:15]] + [a[0] for a in ms4[0:15]])

In [147]:
ranks = {}
avg_scores = []
for i in set([a[0] for a in ms1]):
    k = 1
    avg = 0
    ranks_curr = []
    while ms1[k - 1][0] != i and k < len(ms1):
        k += 1
    ranks_curr.append(k)
    avg += k
    
    k = 1
    while ms2[k - 1][0] != i and k < len(ms2):
        k += 1
    ranks_curr.append(k)
    avg += k
    k = 1
    while ms3[k - 1][0] != i and k < len(ms3):
        k += 1
    ranks_curr.append(k)
    avg += k
    
    k = 1
    while ms4[k - 1][0] != i and k < len(ms4):
        k += 1
    ranks_curr.append(k)
    avg += k
    
    ranks[i] = ranks_curr
    avg_scores.append((float(avg/4), i))

In [148]:
### Women's Finalists for Team:
# Simone Biles (X)
# Jordan Chiles (X)
# Joscelyn Roberson
# Kayla Dicello
# Jada Carey
# Shilese Jones
# Zoe Miller
# Kaliya Lincoln
# Skye Blakely 
# Ashlee Sullivan

In [149]:
len(ms_combined)

17

In [150]:
len(set(womens_data['name']))

33

In [151]:
sorted(avg_scores)

[(1.75, 'Simone BILES'),
 (3.0, 'Jordan CHILES'),
 (3.5, 'Joscelyn ROBERSON'),
 (3.75, 'Jade CAREY'),
 (4.5, 'Zoe MILLER'),
 (5.5, 'Kaliya LINCOLN'),
 (6.5, 'Kayla DICELLO'),
 (7.5, 'Shilese JONES'),
 (9.5, 'Ashlee SULLIVAN'),
 (9.5, 'Skye BLAKELY'),
 (12.25, 'Addison FATTA'),
 (12.25, 'Katelyn JONG'),
 (13.75, 'Nola MATTHEWS'),
 (14.25, 'Katelyn ROSEN'),
 (15.0, 'Jordis EICHMAN'),
 (16.25, 'Leanne WONG'),
 (18.25, 'Charlotte BOOTH'),
 (18.75, 'Tiana SUMANASEKERA'),
 (20.0, 'Elle MUELLER'),
 (20.25, 'Lexi ZEISS'),
 (20.25, 'Myli LEW'),
 (21.75, 'Sunisa LEE'),
 (23.0, 'Brooke PIERSON'),
 (23.5, 'Marissa NEAL'),
 (24.0, 'Levi JUNG-RUIVIVAR'),
 (25.0, 'Amelia DISIDORE'),
 (26.5, 'Alicia ZHOU'),
 (27.5, 'Michelle PINEDA'),
 (28.5, 'Eveylynn LOWE'),
 (29.5, 'Dulcy CAYLOR'),
 (30.75, 'Avery MOLL'),
 (31.75, 'Madray JOHNSON'),
 (32.75, 'Kelise WOOLFORD')]

In [152]:
print([(a, ranks[a]) for a in ms_combined])

[('Zoe MILLER', [7, 1, 5, 5]), ('Jordan CHILES', [2, 5, 2, 3]), ('Leanne WONG', [14, 19, 17, 15]), ('Simone BILES', [1, 4, 1, 1]), ('Shilese JONES', [6, 8, 8, 8]), ('Joscelyn ROBERSON', [3, 6, 3, 2]), ('Skye BLAKELY', [9, 10, 10, 9]), ('Nola MATTHEWS', [13, 16, 13, 13]), ('Kaliya LINCOLN', [8, 2, 6, 6]), ('Katelyn JONG', [12, 14, 11, 12]), ('Sunisa LEE', [30, 11, 14, 32]), ('Jade CAREY', [4, 3, 4, 4]), ('Katelyn ROSEN', [15, 13, 15, 14]), ('Jordis EICHMAN', [16, 12, 16, 16]), ('Addison FATTA', [11, 15, 12, 11]), ('Kayla DICELLO', [5, 7, 7, 7]), ('Ashlee SULLIVAN', [10, 9, 9, 10])]


In [153]:
len(ms_combined)

17

In [158]:
mens_data[mens_data['LastName'] == "LASIC-ELLIS"]


,LastName,FirstName,Gender,Country,Date,Competition,Round,Location,Apparatus,Rank,D_Score,E_Score,Penalty,Score,Nation,name,Year,Month
1076,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,final,"Cairo, Egypt",FX,8.0,5.4,6.966,NaN,12.366,USA,Ian LASIC-ELLIS,2022,3
1077,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,qual,"Cairo, Egypt",FX,2.0,5.7,8.466,NaN,14.166,USA,Ian LASIC-ELLIS,2022,3
1078,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,final,"Cairo, Egypt",HB,4.0,4.9,8.533,NaN,13.433,USA,Ian LASIC-ELLIS,2022,3
1079,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,qual,"Cairo, Egypt",HB,6.0,4.9,8.333,NaN,13.233,USA,Ian LASIC-ELLIS,2022,3
1080,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,qual,"Cairo, Egypt",PB,14.0,5.6,7.866,NaN,13.466,USA,Ian LASIC-ELLIS,2022,3
1081,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,qual,"Cairo, Egypt",PH,21.0,4.5,8.166,NaN,12.666,USA,Ian LASIC-ELLIS,2022,3
1082,LASIC-ELLIS,Ian,m,USA,17-20 Mar 2022,2022 Cairo World Cup,qual,"Cairo, Egypt",SR,12.0,5.0,8.800,NaN,13.800,USA,Ian LASIC-ELLIS,2022,3
1083,LASIC-ELLIS,Ian,m,USA,9-12 June 2022,2022 Osijek World Challenge Cup,qual,"Osijek, Croatia",FX,19.0,5.7,7.300,NaN,13.000,USA,Ian LASIC-ELLIS,2022,6
1084,LASIC-ELLIS,Ian,m,USA,9-12 June 2022,2022 Osijek World Challenge Cup,qual,"Osijek, Croatia",HB,12.0,5.2,7.850,NaN,13.050,USA,Ian LASIC-ELLIS,2022,6
1085,LASIC-ELLIS,Ian,m,USA,9-12 June 2022,2022 Osijek World Challenge Cup,qual,"Osijek, Croatia",PB,10.0,5.4,8.500,NaN,13.900,USA,Ian LASIC-ELLIS,2022,6
